In [1]:
using DataFrames, Feather

In [2]:
@enum FailureType begin
    # General ones
    Unknown
    InferredFailure
    PrintingChange
    TestAmbiguities
    NewAmbiguity
    ApproxError
    DocTest
    BadBoy
    # Specific
    IRToolsParams
    ParameterUpperBound
    ModuleIR
    BoundsErrorCompiler
    Cassette
    OldMbedTLSSyntax
    GPUErrors
end

In [3]:
if !isdir("data")
    run(`tar -xvf data.tar.gz`)
end
primary  = Feather.read("data/primary.feather");
against = Feather.read("data/against.feather");

In [4]:
package_results = join(primary, against,
     on=:uuid, kind=:left, makeunique=true, indicator=:source);

In [5]:
changed_tests = filter(test->test.source == "both" &&
                 test.status != test.status_1, package_results)
fails = changed_tests[changed_tests[!, :status] .== ":fail", :];
sort!(fails , :name);
insert!(fails, size(fails, 2), fill(Unknown, size(fails, 1)), :why);

In [6]:
query(fails, s) = filter(row -> occursin(s, row[:log]), fails)    

function update_reason!(fails, needle, why)
    idxs = findall(row -> occursin(needle, row), fails.log)
    fails.why[idxs] .= why
    return fails
end

total_unknown(fails) = count(x -> x.why == Unknown, eachrow(fails))

total_unknown (generic function with 1 method)

In [7]:
# Generic ones
update_reason!(fails, "Test.detect_ambiguities", TestAmbiguities)
update_reason!(fails, "Expression: all_doctests()", DocTest)
update_reason!(fails, "does not match inferred return type", InferredFailure);
update_reason!(fails, "Expression: isapprox", ApproxError)
update_reason!(fails, "is ambiguous. Candidates:", NewAmbiguity)
update_reason!(fails, "Evaluated: \"", PrintingChange)


# Specific ones for this release
update_reason!(fails, "Params not defined", IRToolsParams);
update_reason!(fails, "parameter_upper_bound", ParameterUpperBound);
update_reason!(fails, "Module IR does not contain specified entry function", ModuleIR);
update_reason!(fails, "Internal error: encountered unexpected error in runtime:\nBoundsError", BoundsErrorCompiler
);
update_reason!(fails, "@ Cassette", Cassette)
update_reason!(fails, "/home/pkgeval/.julia/packages/MbedTLS/", OldMbedTLSSyntax)
update_reason!(fails, "Test.detect_ambiguities", TestAmbiguities)
update_reason!(fails, "unsupported or misplaced expression \"return\" in function", GPUErrors)

# Baddies

# https://github.com/JuliaDiffusionBayes/DiffusionDefinition.jl/issues/11
update_reason!(fails, "DD.remove_curly(Vector{Float64}) == Array", BadBoy)
update_reason!(fails, "Unknown observable eltype", BadBoy)
update_reason!(fails, "ArgumentError: Invalid type: typename(SwingEq)", BadBoy)

nothing

In [8]:
total_unknown(fails)

70

In [9]:
filter(x -> x.why == Unknown, fails).name[20:60]

41-element Array{String,1}:
 "DiffEqBiological"
 "DimensionalData"
 "DomainSets"
 "ExprTools"
 "Fermi"
 "ForneyLab"
 "GPUCompiler"
 "GeoGreensFunctions"
 "Graphene"
 "GslibIO"
 "ImagePhaseCongruency"
 "IncompleteLU"
 "InterProcessCommunication"
 ⋮
 "PhaseSpaceIO"
 "PhysicsInformedML"
 "PushVectors"
 "QuantReg"
 "QuasiArrays"
 "RBNF"
 "RandomBasedArrays"
 "ReferenceTests"
 "Rotations"
 "SMC"
 "Sched"
 "SpatialJackknife"

In [10]:
query(fails, "Log Pattern: ")

,julia,name,uuid
,String,String,String
1,"v""1.6.0-DEV-4978cd5a4f""",DomainSets,"UUID(""5b8099bc-c8ec-5219-889f-1d9e522a28bf"")"
2,"v""1.6.0-DEV-4978cd5a4f""",MLJModels,"UUID(""d491faf4-2d78-11e9-2867-c94bc002c0b7"")"


In [11]:
z = []
for i in instances(FailureType)
    n = count(row -> row.why == i, eachrow(fails))
    push!(z, (n, i))
end
sort!(z; rev=true)

15-element Array{Any,1}:
 (70, Unknown)
 (58, ParameterUpperBound)
 (37, ModuleIR)
 (35, PrintingChange)
 (32, IRToolsParams)
 (15, DocTest)
 (12, Cassette)
 (7, BoundsErrorCompiler)
 (7, NewAmbiguity)
 (5, GPUErrors)
 (5, OldMbedTLSSyntax)
 (3, BadBoy)
 (3, InferredFailure)
 (2, ApproxError)
 (2, TestAmbiguities)